In [12]:
from google import genai
from google.genai import types
import base64
import json
import os
from dotenv import load_dotenv

from pydantic import BaseModel, Field

import unicodedata
import re

In [13]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

The story id, from 1 to 13 according to the 13 Ainu Kamuy Yukars translated by Chiri Yukie. The Yukar ID starts at 1. Chiri's Preface is 0.

In [14]:
start_at = 0
end_at = 0

In [15]:
class Translation(BaseModel):
    """The translation, and the comments"""
    translation: str = Field(description="The translation")
    comment: str = Field(description="Comments on the translation")

In [16]:
system_instruction_prompt = """You are a professional translator. You know Japanese, English and Chinese. You can translate Japanese into either Chinese or English. You can also translate Chinese into English, and English into Chinese."""

client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
    max_output_tokens = 8192,
    response_mime_type = 'application/json',
    response_schema = Translation,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=system_instruction_prompt)],
  )

In [17]:
poetic_translation_prompt = """Translate the following text from English to Chinese. The original text is translation from Japanese which is a prologue. 
Keep the original meaning. Use modern Chinese. Display in Traditional Chinese. Only allow Chinese and English characters.
If a term cannot be translated, keep the original language."""

#descriptive_translation_prompt = "Translate the following text from Japanese to Chinese. Keep the original meanings. Display in Traditional Chinese."

In [18]:
# read the content page of Japanese translation and get the Japanese translated title
with open("Chiri_Japanese_Translation/content.txt", "r", encoding="utf8") as f:
    japanese_content = f.read()
    japanese_content = unicodedata.normalize('NFKC', japanese_content)


s=re.split(r'\n\n', japanese_content)
japanese_titles = re.split(r'\n', s[1])

In [19]:
# read the content page of Ainu original text and get the original title
with open("original_Ainu_text/content.txt", "r", encoding="utf8") as f:
    ainu_content = f.read()
    ainu_content = unicodedata.normalize('NFKC', ainu_content)


s=re.split(r'\n\n', ainu_content)
ainu_titles = re.split(r'\n', s[1])

In [20]:
# Read the markdown template for writing the Chinese translations to Markdown file

#read in the template
with open("templates/raw_output_md_template_cross_lingual", "r", encoding="utf8") as f:
    md_template = f.read()
    md_template = unicodedata.normalize('NFKC', md_template)

In [21]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /,input_text :str, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}\n\n{input_text}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output

def get_output_file_name_key(title :str):
    # setup the output file name
    s = title.split()
    md_name_part = s[0]

    name_2nd_part = ""

    for text in s:
        if text.startswith('“'):
            name_2nd_part = text.replace('“', '').replace('”', '')
        
    md_name_part += "_" + name_2nd_part

    return md_name_part



In [22]:
with open(f"Initial_LLM_prompts_and_translations_main_text/English_Translation_JSON/Chiri_Yukie_prologue.json", "r", encoding="utf8") as f:
    data_dict = json.load(f)

    translation = generate(client,generate_content_config,model,input_text=data_dict['english_translation'],prompt=poetic_translation_prompt)

    translation_dict = json.loads(translation)

    #descriptive_translation = generate(client,generate_content_config,model,input_text=japanese_story,prompt=descriptive_translation_prompt)

    md_output = md_template.format(translated_language="Chinese", original_language="English", ainu_title="",
                                poetic_prompt=poetic_translation_prompt,
                                japanese_title=japanese_titles[0],
                                input_text = data_dict['english_translation'],
                                output_poetic=translation_dict['translation'])

    data_dict['chinese_translation'] = translation_dict['translation']
    data_dict['comment'] = translation_dict['comment']

    with open(f"cross_lingual_LLM_prompts_and_translations_main_text/Chinese_Translation_JSON/Chiri_Yukie_prologue.json", "w", encoding="utf8") as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=4)

    with open(f"cross_lingual_LLM_prompts_and_translations_main_text/Chinese_Translation/Chiri_Yukie_prologue_to_Chinese.md", "w", encoding="utf8") as f:
        f.write(md_output)



{
  "comment": "Translated from English to Chinese (Traditional). Kept original meaning and used modern Chinese. Some terms that are difficult to translate directly are kept in English.",
  "translation": "## 序\n\n很久以前，這片廣闊的北海道土地是我們祖先自由的國度。他們像天真的孩子一樣，被美麗的大自然擁抱，悠閒快樂地生活著。 真的，他們是大自然的寵兒，是如此幸福的人們。\n\n在冬天，他們踢開覆蓋森林的厚厚積雪，冒著冰冷的空氣，穿越一座又一座山去獵熊。 在夏天，他們在綠色的波浪上航行著葉子般的小船，涼爽的微風在其中游動，白鷗的歌聲是他們的伴侶，整天釣魚。 在百花盛開的春天，他們沐浴在柔和的陽光下，與永遠鳴叫的鳥兒一起歌唱，採集蜂斗菜和艾蒿。 在深紅色的秋天，他們分開芒草田，芒草的穗在秋風中搖曳，直到捕鮭魚的篝火在傍晚消失。 鹿的叫聲在山谷中迴盪，他們在圓月下做夢。 啊，多麼快樂的生活啊！\n\n那個和平的世界現在已經成為過去。 自那些夢想破滅以來，已經過去了幾十年。 這片土地經歷了快速的轉變，山脈和原野逐漸讓位於村莊，村莊讓位於城鎮。\n\n在我們還沒意識到的時候，大自然古老的面貌已經褪色，曾經在田野和山脈中快樂生活的人們都到哪裡去了？ 我們為數不多的族人只能驚訝地看著這個變化中的世界。 然而，那些古代人美麗的靈魂，他們的一舉一動都受到宗教情感的支配，現在已經從他們的眼中消失了，取而代之的是焦慮和怨恨，變得遲鈍和陰暗，無法看到前進的方向。 他們必須依靠別人的憐憫，真是可悲的景象。 哦，那些正在消亡的人……那就是我們現在的名字。 我們背負著多麼悲傷的名字啊。\n\n過去，我們幸福的祖先永遠無法想像他們的家園有一天會淪落到如此悲慘的境地。\n\n時間不停地流逝，世界無限地前進。 在我們這些正在這激烈的競爭舞台上暴露失敗者醜態的人中，也許有一天會出現兩三個堅強的個體，最終有一天我們可以跟上這個前進的世界。 這確實是我們熱切的希望，我們日夜為此祈禱。\n\n但是……我們敬愛的祖先過去每天用來互相交流的那麼多語言，他們傳承下來的古老而美麗的文字，會不會隨著消亡的弱者